# From DDPG to SAC

This tutorial will introduce DDPG, TD3 as well as SAC.

## Helper function

In [17]:
import numpy as np
import scipy.signal
from copy import deepcopy
import itertools
import pathlib
import gym
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal
from torch.optim import Adam
import wandb  # for logging

In [18]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)

def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

def count_vars(module):
    """ count parameters number of module"""
    return sum([np.prod(p.shape) for p in module.parameters()])

## Model

In [19]:
# actor
class SquashedGaussianMLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        self.net = mlp([obs_dim] + list(hidden_sizes), activation, activation)
        self.mu_layer = nn.Linear(hidden_sizes[-1], act_dim)
        self.log_std_layer = nn.Linear(hidden_sizes[-1], act_dim)
        self.act_limit = act_limit
        # TODO: notice this limitation
        self.LOG_STD_MAX = 2
        self.LOG_STD_MIN = -20

    def forward(self, obs, deterministic=False, with_logprob=True):
        net_out = self.net(obs)
        mu = self.mu_layer(net_out)
        log_std = self.log_std_layer(net_out)
        log_std = torch.clamp(log_std, self.LOG_STD_MIN, self.LOG_STD_MAX)
        std = torch.exp(log_std)

        # Pre-squash distribution and sample
        pi_distribution = Normal(mu, std)
        if deterministic:
            # Only used for evaluating policy at test time.
            pi_action = mu
        else:
            pi_action = pi_distribution.rsample()  # shape [batch, act_dim], e.g.[100, 6]

        if with_logprob:
            # TODO: Checkout this
            # Compute logprob from Gaussian, and then apply correction for Tanh squashing.
            # NOTE: The correction formula is a little bit magic. To get an understanding 
            # of where it comes from, check out the original SAC paper (arXiv 1801.01290) 
            # and look in appendix C. This is a more numerically-stable equivalent to Eq 21.
            # Try deriving it yourself as a (very difficult) exercise. :)
            logp_pi = pi_distribution.log_prob(pi_action).sum(axis=-1)  # shape [batch] ,e.g [100]
            logp_pi -= (2*(np.log(2) - pi_action - F.softplus(-2*pi_action))).sum(axis=1) # sum(axis=1) --> over act_dim
        else:
            logp_pi = None

        pi_action = torch.tanh(pi_action)  # squash it to [-1, 1]
        pi_action = self.act_limit * pi_action

        return pi_action, logp_pi

# Q function
class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

# actor-critic
class MLPActorCritic(nn.Module):
    """
    ===========  ================  ======================================
    Call         Output Shape      Description
    ===========  ================  ======================================
    ``act``      (batch, act_dim)  | Numpy array of actions for each 
                                   | observation.
    ``q1``       (batch,)          | Tensor containing one current estimate
                                   | of Q* for the provided observations
                                   | and actions. (Critical: make sure to
                                   | flatten this!)
    ``q2``       (batch,)          | Tensor containing the other current 
                                   | estimate of Q* for the provided observations
                                   | and actions. (Critical: make sure to
                                   | flatten this!)
    ===========  ================  ======================================

    Calling ``pi`` should return:
    ===========  ================  ======================================
    Symbol       Shape             Description
    ===========  ================  ======================================
    ``a``        (batch, act_dim)  | Tensor containing actions from policy
                                   | given observations.
    ``logp_pi``  (batch,)          | Tensor containing log probabilities of
                                   | actions in ``a``. Importantly: gradients
                                   | should be able to flow back into ``a``.
    ===========  ================  ======================================        

    """

    def __init__(self, observation_space, action_space, hidden_sizes=(256,256), activation=nn.ReLU):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]
#         print("obs_dim", obs_dim, "act_dim", act_dim, "act_limit", act_limit)
        # build policy and value functions
        self.pi = SquashedGaussianMLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit)
        self.q1 = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation)
        self.q2 = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation)

    def act(self, obs, deterministic=False):
        with torch.no_grad():
            a, _ = self.pi(obs, deterministic, False)
            return a.numpy()

## Agent

In [24]:
class SacAgent():
    """ update model and take action. """
    def __init__(self, observation_space, action_space, config, hidden_sizes=(256,256), activation=nn.ReLU):

        self.polyak = config.polyak
        self.alpha = config.alpha
        self.gamma = config.gamma
        # Create actor-critic module and target networks
        self.ac = MLPActorCritic(observation_space, action_space, hidden_sizes=(256,256), activation=nn.ReLU)
        self.ac_targ = deepcopy(self.ac)

        # Freeze target networks with respect to optimizers (only update via polyak averaging)
        for p in self.ac_targ.parameters():
            p.requires_grad = False

        # List of parameters for both Q-networks (save this for convenience)
        self.q_params = itertools.chain(self.ac.q1.parameters(), self.ac.q2.parameters())
        
        # setup optimizer
        self.pi_optimizer = Adam(self.ac.pi.parameters(), lr=config.lr)
        self.q_optimizer = Adam(self.q_params, lr=config.lr)
        
        # Count variables (protip: try to get a feel for how different size networks behave!)
        var_counts = tuple(count_vars(module) for module in [self.ac.pi, self.ac.q1, self.ac.q2])
        print('\nNumber of parameters: \t pi: %d, \t q1: %d, \t q2: %d\n'%var_counts)

    # Set up function for computing SAC Q-losses
    def _compute_loss_q(self, data):
        o, a, r, o2, d = data['obs'], data['act'], data['rew'], data['obs2'], data['done']

        q1 = self.ac.q1(o,a)
        q2 = self.ac.q2(o,a)

        # Bellman backup for Q functions
        with torch.no_grad():
            # Target actions come from *current* policy # one key different between TD3 and SAC
            a2, logp_a2 = self.ac.pi(o2)

            # Target Q-values
            q1_pi_targ = self.ac_targ.q1(o2, a2)
            q2_pi_targ = self.ac_targ.q2(o2, a2)
            q_pi_targ = torch.min(q1_pi_targ, q2_pi_targ)  
            backup = r + self.gamma * (1 - d) * (q_pi_targ - self.alpha * logp_a2) 
            
        # MSE loss against Bellman backup
        loss_q1 = ((q1 - backup)**2).mean()
        loss_q2 = ((q2 - backup)**2).mean()
        loss_q = loss_q1 + loss_q2

        # Useful info for logging
        q_info = dict(Q1Vals=q1.detach().numpy(),
                      Q2Vals=q2.detach().numpy())

        return loss_q, q_info

    # Set up function for computing SAC pi loss
    def _compute_loss_pi(self, data):
        o = data['obs']
        pi, logp_pi = self.ac.pi(o)
        q1_pi = self.ac.q1(o, pi)
        q2_pi = self.ac.q2(o, pi)
        q_pi = torch.min(q1_pi, q2_pi)

        # Entropy-regularized policy loss
        loss_pi = (self.alpha * logp_pi - q_pi).mean()

        # Useful info for logging
        pi_info = dict(LogPi=logp_pi.detach().numpy())

        return loss_pi, pi_info

    def update(self, data):
        # First run one gradient descent step for Q1 and Q2
        self.q_optimizer.zero_grad()
        loss_q, q_info = self._compute_loss_q(data)
        loss_q.backward()
        self.q_optimizer.step()
        
        wandb.log({"LossQ": loss_q.item(), **q_info})

        # Freeze Q-networks so you don't waste computational effort 
        # computing gradients for them during the policy learning step.
        for p in self.q_params:
            p.requires_grad = False

        # Next run one gradient descent step for pi.
        self.pi_optimizer.zero_grad()
        loss_pi, pi_info = self._compute_loss_pi(data)
        loss_pi.backward()
        self.pi_optimizer.step()

        # Unfreeze Q-networks so you can optimize it at next DDPG step.
        for p in self.q_params:
            p.requires_grad = True

        wandb.log({"LossPi": loss_pi.item(), **pi_info})

        # Finally, update target networks by polyak averaging.
        with torch.no_grad():
            for p, p_targ in zip(self.ac.parameters(), self.ac_targ.parameters()):
                p_targ.data.mul_(self.polyak)
                p_targ.data.add_((1 - self.polyak) * p.data)

    def get_action(self, o, deterministic=False):
        return self.ac.act(torch.as_tensor(o, dtype=torch.float32), 
                      deterministic)


## Replay Buffer

In [21]:
class ReplayBuffer:
    """
    A simple FIFO experience replay buffer for SAC agents.
    """

    def __init__(self, obs_dim, act_dim, size):
        self.obs_buf = np.zeros(combined_shape(size, obs_dim), dtype=np.float32)
        self.obs2_buf = np.zeros(combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(combined_shape(size, act_dim), dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.ptr, self.size, self.max_size = 0, 0, size

    def store(self, obs, act, rew, next_obs, done):
        self.obs_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample_batch(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size)
        batch = dict(obs=self.obs_buf[idxs],
                     obs2=self.obs2_buf[idxs],
                     act=self.act_buf[idxs],
                     rew=self.rew_buf[idxs],
                     done=self.done_buf[idxs])
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in batch.items()}

## Training and testing

In [22]:
if __name__ == '__main__':
        
    # setup hyperparameter
    wandb.init(project="sac")
    config = wandb.config
    config.logdir = pathlib.Path(".")
    config.exp_name = "sac" 
    config.env = "HalfCheetah-v2"
    config.hid = 256  # Number of hidden unit
    config.l = 2  #
    config.gamma = 0.99  # Discount factor. (Always between 0 and 1.)
    config.seed = 0  # Random seed
    config.epochs = 50  # Number of epoch to run and train agent. 
    config.replay_size=int(1e6)  # Maximum length of replay buffer.
    config.polyak=0.995  # Interpolation factor in polyak averaging for target networks.
    config.lr=1e-3  # learning rate, here we use the same lr for all q and pi
    config.alpha=0.2  # Entropy regularization coefficient. (Equivalent to inverse of reward scale in the original SAC paper.)
    config.batch_size=100  # Minibatch size for SGD.
    
    config.steps_per_epoch = 4000  # Number of steps of interaction (state-action pairs) for the agent and 
                                   # the environment in each epoch.
    config.start_steps=10000  # pre collection steps (with random policy) before using real policy
    config.update_after=1000  # Number of env interactions to collect before starting to do gradient descent updates.
    config.update_every=50  # Number of env interactions that should elapse between gradient descent updates. 
                            # Note: Regardless of how long you wait between updates, 
                            # the ratio of env steps to gradient steps is locked to 1.
    config.num_test_episodes=10 # Number of episodes to test the deterministic policy at the end of each epoch.
    config.max_ep_len=1000,  # Maximum length of trajectory / episode / rollout.
    config.save_freq = 1  # How often (in terms of gap between epochs) to save the current policy and value function.
    
    # setup random seed and num_threads
    torch.manual_seed(config.seed)
    np.random.seed(config.seed)
    torch.set_num_threads(torch.get_num_threads())

    # setup env
    env, test_env = gym.make(config.env), gym.make(config.env)
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape[0]
    act_limit = env.action_space.high[0]
    
    # setup replay beffer
    replay_buffer = ReplayBuffer(obs_dim=obs_dim, act_dim=act_dim, size=config.replay_size)
    
    # setup agent
    agent = SacAgent(env.observation_space, 
                     env.action_space, config, 
                     hidden_sizes=[config.hid]*config.l)
    
    wandb.watch(agent.ac, log="all")
    def test_agent():
        for j in range(config.num_test_episodes):
            o, d, ep_ret, ep_len = test_env.reset(), False, 0, 0
            while not(d or (ep_len == config.max_ep_len)):
                # Take deterministic actions at test time 
                o, r, d, _ = test_env.step(agent.get_action(o, True))
                ep_ret += r
                ep_len += 1
            wandb.log({"TestEpRet": ep_ret, "TestEpLen": ep_len})

    # Prepare for interaction with environment
    total_steps = config.steps_per_epoch * config.epochs
    start_time = time.time()
    o, ep_ret, ep_len = env.reset(), 0, 0  # to record total reward of an epiosde; to track the steps

    # Main loop: collect experience in env and update/log each epoch
    for t in range(total_steps):
        
        # Until start_steps have elapsed, randomly sample actions
        # from a uniform distribution for better exploration. Afterwards, 
        # use the learned policy. 
        if t > config.start_steps:
            a = agent.get_action(o)
        else:
            a = env.action_space.sample()

        # Step the env
        o2, r, d, _ = env.step(a)
        print(r)
        ep_ret += r
        ep_len += 1

        # Ignore the "done" signal if it comes from hitting the time
        # horizon (that is, when it's an artificial terminal signal
        # that isn't based on the agent's state)
        d = False if ep_len==config.max_ep_len else d

        # Store experience to replay buffer
        replay_buffer.store(o, a, r, o2, d)

        # Super critical, easy to overlook step: make sure to update 
        # most recent observation!
        o = o2

        # End of trajectory handling
        if d or (ep_len == config.max_ep_len):
            
            wandb.log({"EpRet": ep_ret, "EpLen": ep_len})
            o, ep_ret, ep_len = env.reset(), 0, 0

        # Update handling
        if t >= config.update_after and t % config.update_every == 0:
            for j in range(config.update_every): # the ratio of env steps to gradient steps is locked to 1.
                batch = replay_buffer.sample_batch(config.batch_size)
                agent.update(data=batch)

        # End of epoch handling
        if (t+1) % config.steps_per_epoch == 0:
            epoch = (t+1) // config.steps_per_epoch

            # Save model
            if (epoch % config.save_freq == 0) or (epoch == config.epochs):
                torch.save(agent.ac.state_dict(), "model.h5")
                wandb.save("model.h5")
            # Test the performance of the deterministic version of the agent.
            test_agent()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


EpRet,-218.63475
EpLen,1000
_step,1562
_runtime,35
_timestamp,1602063378
LossQ,0.56363
LossPi,-3.19117


EpRet,▁
EpLen,▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
LossQ,█▆▄▄▃▆▂▃▄▄▃▂▆▄▂▃▃▁▃▂▃▂▃▁▄▃▁▂▃▂▁▂▂▃▃▁▅▂▂▃
LossPi,███▇▇▆▆▆▆▆▆▆▆▅▅▅▆▅▅▅▅▅▅▄▄▄▅▄▄▃▄▃▃▃▂▂▂▂▂▁



Number of parameters: 	 pi: 73484, 	 q1: 72193, 	 q2: 72193

-0.36851502633411826
-0.4976370747708496
-0.15568770838938634
-1.0588110748825816
-0.3220470987497111
0.8354908011847739
0.7433345963697348
-0.9262950499570516
-0.5643416663711207
1.0369046190776046
0.3737668291633721
-0.4983568111600949
-0.45073910806276457
-1.0481813961297999
-0.5210710184961562
0.006635167067612281
-0.08764478603667131
-0.6246170148138088
-0.20279740014014053
-0.3797967883811328
0.014401149817588693
-0.6446747947121975
-0.5864804743223372
0.4727658681720449
0.22425884639700472
0.041472569940256956
-0.7030183992172712
-0.09625165176655232
0.3536536670775039
-0.3927887038869582
-0.5653668511812988
0.4685075344292875
-0.2324226178333643
-0.4667005112526471
-0.3506257035403071
-1.0291789715830193
0.3576039122574591
-0.30790974420115863
-0.3589613251033672
-0.24483102398005568
-0.051153244445577034
-1.2013326396477106
-0.8783783851422492
-0.49891050226300193
0.05086866981421162
0.3828469752304688
-0.5644643400

-0.17192109276608605
-0.8862922889509377
-1.0754304228322886
-0.9297872813531057
-0.8933470144241845
-0.4712467027548598
-0.7459529943813734
-0.812098624845337
-0.4361008449037442
-0.5765902295378936
-0.7113597033807089
-1.0106199779035236
-1.063504975898225
-0.743314711085192
-0.818571688202067
-0.9381119948679306
-1.0383704046849602
-0.38114053393351033
-0.5805952579260001
-1.510449219040074
-0.8222352041055118
-1.003278698560878
-0.7794220710170642
-0.7528075786630751
-0.06935744161402013
-0.7256825816750152
-0.2474706492414967
0.31794960200452704
-0.014466408132584985
-0.801766023386574
-0.046014026565074306
-0.05530522178192074
0.21033127046078945
0.8900867104794209
0.7320245797997855
0.35607320061508146
-0.6202256584207562
-0.4533971965704254
-0.4345346199507894
-0.5696752441679017
0.11297695399981933
-0.04808430450079301
0.44784205008818534
0.2610118745142259
-0.48871417227273073
-0.2133316835980814
0.1743150322156154
0.1510146936140906
0.12720337865180298
0.43434347306557225
-0

...

0.28007284856439724
0.24605031316552511
-0.5985145503645839
-0.5142541689399818
-0.7226332730093024
-0.7161885711790361
0.1906641370143003
-0.08609239342794625
-1.0700479925512716
-0.2820522053493722
0.2522665516172722
0.10074389847177478
-0.31082632308105645
-0.26682151411610205
0.47940133142581337
0.5077768709702166
0.07144942107704677
-0.22194383266817397
-0.6708383510928861
-0.988920286627089
0.7365307823269156
1.6721384235577037
1.0233911901815447
-0.07664066499867239
-0.7398107268039503
-0.016332592043664496
0.47568848721419704
-0.07258193743021196
-1.5332589984135332
-0.8058067948612588
0.5812860193336903
1.3099554031060712
0.26792159962422735
-0.5207726119029171
0.9063038597776313
1.0621807240346322
0.39287348975450553
0.2973107067896844
0.6269198239032864
0.07198806156292717
0.27658958677853923
0.9487163255429898
1.1649687457521736
0.19100018295413043
-0.40896711522136897
-0.6519702924698884
0.5403961052052555
0.5936609103496735
0.2136432292452518
0.000896996809271744
-0.20669

-0.45911815318228194
-0.28424019428113073
0.05950218538578653
-0.44789638809541116
-0.021181475682870862
-0.14696518769635888
-0.372243662649263
-0.6600751374810678
0.17041015915114882
0.5482141680162862
0.2928978960751543
-0.2986605387933977
0.3897831463722504
0.1125807247434295
0.4792548781395546
0.2479526100760708
-1.0398254930786905
0.03556097840756962
0.818832351712117
0.23353044437640946
-0.656563812409722
0.4210072041975785
0.8614867609506505
0.056416438175250494
-0.12000257994516306
0.1583991737903574
0.5753925788345136
0.42546184167676415
0.5307992652212036
0.031213256870178424
-0.24322913828737003
0.23671505596679124
1.1454933934655436
-0.5194444208774733
-0.787675427044352
0.07079366393654637
0.26808794174766815
0.5201808268895327
0.48376202934396917
0.41482122598093407
0.08775760119958276
-0.9915293837211969
-0.5609706471221806
0.19735561339943497
-0.08345673897746622
-0.07893251589188566
-0.7090657480882343
-1.509781531918233
-0.6164640657074962
0.3127778705230575
-0.00709

-0.23533713479486967
0.15911212939470898
-0.8373620986694364
-0.7019600664896247
-0.8059521227647544
-0.5590714116555914
0.7116393862402146
0.008954886204509183
-0.38515842129435446
-0.4896623258213145
-1.1125461700681667
-0.6704105473932713
-0.34961879856522293
-0.3563380014751191
-0.4455460521647108
-0.5367290003776459
-0.26808030092338075
-0.6336902765556826
-1.281034052941646
-0.581570623939623
-0.3458337527399314
-1.1443275484558435
-1.4304527201395199
-0.8209874623867514
-0.4547869279065237
0.021044978920055413
-1.2580462911956816
-0.6284184701460939
0.03686781214925272
-0.9758070560236116
-1.6954163129740307
-0.5936715351597919
0.11213042339271978
-1.1884015729782322
-1.91687566822904
-1.5032025371187905
-1.1071692719900375
-0.3635215052014896
-0.31127415515985374
-1.678459407953115
-1.4790269386624266
-1.3801940751832915
-0.8975134699344132
-0.405163389061854
-0.6979263303238317
-0.8507158485580433
-0.7039485723810387
-0.9997851993344047
-0.6098433626435558
-0.3823626415098296


-0.8387144394205173
-1.6727899875053354
-0.8423673293790493
0.21005774144913258
-0.4898588273002993
-1.7013418085452379
-0.16637877389376995
-0.17877785536233265
-0.3554885429851936
-1.1679731941370903
-1.0209676429856942
0.3617798559583633
0.8304278948061394
1.2227592894800616
0.7579720858952712
0.45380316696162026
-0.5571106865680988
0.5948381445242127
1.9149457225276691
1.0563842903159475
0.6097543891391588
1.3146618051277301
1.0845200558243064
0.7195530593201638
-0.26249926246167093
-1.0024474286793712
0.5698498203152211
0.6159884987570812
-1.446053968049189
-1.0879210717565662
0.8050825093147985
1.2421168286344568
0.8231146658935089
0.024370617880685286
0.1748166830176821
1.462163466048834
1.7299054066378532
1.3975417620108828
0.21886898446278236
0.3328141035404954
1.5138425840822456
1.4091118958652196
0.2452915926769691
0.8355105315821632
1.9451287939080948
2.846485554360045
1.8396214846876524
0.33536147668522887
1.0368015405036615
2.2011698571383844
1.281428530608661
1.116588868

KeyboardInterrupt: 